## First, let's install and import all required libraries

In [1]:
!pip install -r requirements.txt

In [2]:
import collections                        #This library adds some extras to the standard python data structures
import folium                             #Great library for plotting data on maps
import json                               #This library allows us to load and hyndle json files with python
import os                                 #This module is usefull for handling paths and directories
import webbrowser                         #This library allows to handle the webbrowser from python
import pandas as pd
from folium.plugins import HeatMap
from progressbar import ProgressBar, Bar, ETA, Percentage

## Now, let's load and analyze our location history json file 

In [3]:
data = json.load(open('Location_History_Juan.json'))

In [4]:
data

{'locations': [{'timestampMs': '1354262153219',
   'latitudeE7': 524535915,
   'longitudeE7': 132941230,
   'accuracy': 32},
  {'timestampMs': '1354271661093',
   'latitudeE7': 524533301,
   'longitudeE7': 132934986,
   'accuracy': 32},
  {'timestampMs': '1354287407617',
   'latitudeE7': 524533301,
   'longitudeE7': 132934986,
   'accuracy': 32},
  {'timestampMs': '1354291299371',
   'latitudeE7': 524534106,
   'longitudeE7': 132921642,
   'accuracy': 62},
  {'timestampMs': '1354291359891',
   'latitudeE7': 524537807,
   'longitudeE7': 132911611,
   'accuracy': 52,
   'activity': [{'timestampMs': '1354291328356',
     'activity': [{'type': 'STILL', 'confidence': 100}]}]},
  {'timestampMs': '1354291420670',
   'latitudeE7': 524537807,
   'longitudeE7': 132911611,
   'accuracy': 52},
  {'timestampMs': '1354291600666',
   'latitudeE7': 524535330,
   'longitudeE7': 132926220,
   'accuracy': 55,
   'activity': [{'timestampMs': '1354291525216',
     'activity': [{'type': 'STILL', 'confidence

In [5]:
type(data)

dict

In [6]:
data["locations"]

[{'timestampMs': '1354262153219',
  'latitudeE7': 524535915,
  'longitudeE7': 132941230,
  'accuracy': 32},
 {'timestampMs': '1354271661093',
  'latitudeE7': 524533301,
  'longitudeE7': 132934986,
  'accuracy': 32},
 {'timestampMs': '1354287407617',
  'latitudeE7': 524533301,
  'longitudeE7': 132934986,
  'accuracy': 32},
 {'timestampMs': '1354291299371',
  'latitudeE7': 524534106,
  'longitudeE7': 132921642,
  'accuracy': 62},
 {'timestampMs': '1354291359891',
  'latitudeE7': 524537807,
  'longitudeE7': 132911611,
  'accuracy': 52,
  'activity': [{'timestampMs': '1354291328356',
    'activity': [{'type': 'STILL', 'confidence': 100}]}]},
 {'timestampMs': '1354291420670',
  'latitudeE7': 524537807,
  'longitudeE7': 132911611,
  'accuracy': 52},
 {'timestampMs': '1354291600666',
  'latitudeE7': 524535330,
  'longitudeE7': 132926220,
  'accuracy': 55,
  'activity': [{'timestampMs': '1354291525216',
    'activity': [{'type': 'STILL', 'confidence': 100}]}]},
 {'timestampMs': '1354291661872'

In [7]:
df = pd.DataFrame(data["locations"])

In [8]:
df = df.drop(columns=['activity'])
df

,timestampMs,latitudeE7,longitudeE7,accuracy
0,1354262153219,524535915,132941230,32
1,1354271661093,524533301,132934986,32
2,1354287407617,524533301,132934986,32
3,1354291299371,524534106,132921642,62
4,1354291359891,524537807,132911611,52
...,...,...,...,...
205292,1500397386151,481479476,115870654,10
205293,1500404117256,481168069,115342124,10
205294,1500407503014,481146421,115321200,41
205295,1500407508999,481144178,115320414,10


## Now we can extract and tranform the data we need to make our map 

In [9]:
coordinates = collections.defaultdict(int)

In [10]:
coordinates

defaultdict(int, {})

In [11]:
max_coordinates = (0, 0)

In [12]:
max_magnitude = 0

In [13]:
"""Here you transfor the coordinates given by google in actual longitude and latitude coordinates and
show the progress using the progress bar widget"""

w = [Bar(), Percentage(), " ", ETA()]
with ProgressBar(max_value=len(data["locations"]), widgets=w) as pb:
    for i, loc in enumerate(data["locations"]):
        # print(i)
        # print(loc)
        if "latitudeE7" not in loc or "longitudeE7" not in loc:
            continue
        coords = (round(loc["latitudeE7"] / 1e7, 6),
                    round(loc["longitudeE7"] / 1e7, 6))
        # print(coords)

        
        """Here you calculate the magnitude for all coordinates"""
        
        #print(loc["timestampMs"])
        coordinates[coords] += 1 #these are the magnitude values we will need for the coordinates dictionary
        #print(coordinates[coords])
            
        if coordinates[coords] > max_magnitude:
            max_coordinates = coords
            max_magnitude = coordinates[coords]
            #print(max_coordinates)
            #print(max_magnitude)
        
        """This command updates the progress bar"""
        pb.update(i)

|##########################################################|100% Time:  0:00:02


In [14]:
coordinates

defaultdict(int,
            {(52.453592, 13.294123): 1,
             (52.45333, 13.293499): 2,
             (52.453411, 13.292164): 1,
             (52.453781, 13.291161): 2,
             (52.453533, 13.292622): 26,
             (52.460155, 13.309098): 1,
             (52.453497, 13.292151): 4,
             (52.454766, 13.295345): 1,
             (52.453444, 13.292243): 1,
             (52.45346, 13.29226): 2,
             (52.454775, 13.295338): 1,
             (52.453617, 13.292921): 1,
             (52.453981, 13.293744): 1,
             (52.454087, 13.293555): 3,
             (52.453534, 13.292398): 1,
             (52.454838, 13.295468): 3,
             (52.453503, 13.292356): 1,
             (52.45476, 13.29534): 1,
             (52.453521, 13.293006): 3,
             (52.453485, 13.292646): 1,
             (52.454788, 13.295319): 2,
             (52.453521, 13.29396): 2,
             (52.456726, 13.296646): 1,
             (52.463144, 13.304333): 1,
             (52.465033, 13.

In [15]:
tilesoptions = ["openstreetmap", "StamenTerrain", "stamentoner",
                "stamenwatercolor", "cartodbpositron", "cartodbdark_matter"]

tiles = tilesoptions[5]
zoom_start = 10
radius = 7
blur = 4
min_opacity = 0.2
max_zoom = 10


In [16]:
map_data = [(coords[0], coords[1], magnitude)
            for coords, magnitude in coordinates.items()] #coords are keys in the dict. magnitudes are the values

In [17]:
map_data

[(52.453592, 13.294123, 1),
 (52.45333, 13.293499, 2),
 (52.453411, 13.292164, 1),
 (52.453781, 13.291161, 2),
 (52.453533, 13.292622, 26),
 (52.460155, 13.309098, 1),
 (52.453497, 13.292151, 4),
 (52.454766, 13.295345, 1),
 (52.453444, 13.292243, 1),
 (52.45346, 13.29226, 2),
 (52.454775, 13.295338, 1),
 (52.453617, 13.292921, 1),
 (52.453981, 13.293744, 1),
 (52.454087, 13.293555, 3),
 (52.453534, 13.292398, 1),
 (52.454838, 13.295468, 3),
 (52.453503, 13.292356, 1),
 (52.45476, 13.29534, 1),
 (52.453521, 13.293006, 3),
 (52.453485, 13.292646, 1),
 (52.454788, 13.295319, 2),
 (52.453521, 13.29396, 2),
 (52.456726, 13.296646, 1),
 (52.463144, 13.304333, 1),
 (52.465033, 13.305599, 1),
 (52.466943, 13.306098, 1),
 (52.46946, 13.310424, 1),
 (52.470113, 13.31617, 1),
 (52.471311, 13.318864, 2),
 (52.474399, 13.323022, 1),
 (52.478211, 13.328271, 1),
 (52.481909, 13.329606, 1),
 (52.48774, 13.333382, 1),
 (52.488589, 13.339848, 1),
 (52.489958, 13.339426, 2),
 (52.489949, 13.339491, 1),


In [18]:
# Generate background map
m = folium.Map(location=max_coordinates,
                zoom_start=zoom_start,
                tiles=tiles)

# Generate heat map
heatmap = HeatMap(map_data,
                    max_val=max_magnitude,
                    min_opacity=min_opacity,
                    radius=radius,
                    blur=blur,
                    max_zoom=max_zoom)

m.add_child(heatmap)

## Finally, we can save our map as an html file and launch it in the browser 

In [19]:
output_file = tiles + 'heatmapJuan.html'

In [20]:
m.save(output_file)

In [21]:
webbrowser.open("file://" + os.path.realpath(output_file))

True